#### Using SimpleElastix register all non class 1 CT scans and attempt to normalize them into a **healthy** CT scan.

In [162]:
import sys
import os
import csv
from collections import Counter
from configparser import ConfigParser
from glob import glob
import SimpleITK as sitk # pip install SimpleITK
from tqdm import tqdm # pip install tqdm
import numpy as np
import matplotlib as plt
%matplotlib inline

In [77]:
# get file paths and globals from config file, located in utils/ dir
config = ConfigParser()
config.read('../utils/lung_template_config.ini') #local just for now (need if - else for AWS)
LUNA_PATH = config.get('local', 'LUNA_PATH')
TOY_LUNA_PATH = config.get('local', 'TOY_LUNA_PATH')
UID_CSV = config.get('local', 'CSV_PATH')
IMG_SAVE = config.get('local', 'IMG_SAVE')


In [53]:
# create list of CT scan seriesuids where there is no nodule class of 1
unique_ct = Counter({})
with open(UID_CSV) as doc:
    lines = csv.reader(doc)
    next(lines)
    for row in lines:
        if row[0] in unique_ct:
            unique_ct[row[0]] += int(row[1]) #summing nodules per scan
        else:
            unique_ct[row[0]] = int(row[1])
print("Unique CT Scans: " + str(len(unique_ct)))

# remove any UIDs that have a positive nodule
class0_ct = {k:v for k, v in unique_ct.items() if v == 0}
print("Unique CT Scans with no class 1 nodules: " + str(len(class0_ct)))


Unique CT Scans: 76
Unique CT Scans with no class 1 nodules: 28


### Playing with SimpleElastix...

In [180]:
# Simple Test Case - my 1st registration!

mhd_one = '1.3.6.1.4.1.14519.5.2.1.6279.6001.221945191226273284587353530424.mhd'
mhd_two = '1.3.6.1.4.1.14519.5.2.1.6279.6001.249314567767437206995861966896.mhd'

resultImage = sitk.Elastix(sitk.ReadImage(TOY_LUNA_PATH + mhd_two), \
                           sitk.ReadImage(TOY_LUNA_PATH + mhd_one), \
                           "translation")
sitk.WriteImage(resultImage[25], IMG_SAVE+'result1.tif')

In [181]:
# More complex OO Test Case - my 2nd registration!

fixedImage = sitk.ReadImage(TOY_LUNA_PATH + mhd_two)
movingImage = sitk.ReadImage(TOY_LUNA_PATH + mhd_one)
parameterMap = sitk.GetDefaultParameterMap('translation')

elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(movingImage)
elastixImageFilter.SetParameterMap(parameterMap)
elastixImageFilter.Execute()

resultImage = elastixImageFilter.GetResultImage()
transformParameterMap = elastixImageFilter.GetTransformParameterMap()
sitk.WriteImage(resultImage[300], IMG_SAVE+'result2.tif')

Elastix introduces the concept of a parameter map to configure the registration procedure. A parameter map is a collection of key-value pairs that atomically defines the components of the registration and any settings they might require. Only input images and output options need to be specified seperately. Elastix will read a given parameter map and load the specified components at runtime. For Example:

    (Transform "TranslationTransform")
    (NumberOfParameters 3)
    (TransformParameters -10.835814 -12.121919 2.460902)
    (InitialTransformParametersFileName "NoInitialTransform")
    (HowToCombineTransforms "Compose")

In [120]:
# More complex w/ custom parameterMap - my 3rd registration!

parameterMap = sitk.GetDefaultParameterMap('translation')

# Use a non-rigid transform instead of a translation transform
parameterMap['Transform'] = ['BSplineTransform']

# Because of the increased complexity of the b-spline transform,
# it is a good idea to run the registration a little longer to
# ensure convergence
parameterMap['MaximumNumberOfIterations'] = ['512']

resultImage = sitk.Elastix(sitk.ReadImage(TOY_LUNA_PATH + mhd_one), \
                           sitk.ReadImage(TOY_LUNA_PATH + mhd_two), \
                           parameterMap)
sitk.WriteImage(resultImage[300], IMG_SAVE+'result4.tif')

# Print the parameterMap (also gets written to .txt file in active dir)
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.PrintParameterMap()

The resulting transform is called a composite transform since the final transformation is a composition of sequentially applied deformation fields. For example, a non-rigid registration is often initialized with an affine transformation (translation, scale, rotation, shearing) to bring the objects into rough alignment. This makes the registration less suscetible to local minima.

For more info on the main parameters see:
https://simpleelastix.readthedocs.io/ParameterMaps.html#important-parameters

Parameter database from published articles, ones listed for Lung can be used as a baseline approach. Uploading these parameter files is fairly easy when running ct scans through SimpleElastix.
**http://elastix.bigr.nl/wiki/index.php/Parameter_file_database**

##### Some Key Transformations:

- A **rigid transform** can register objects that are related by rotation and translation. 
- The **affine transform** allows for shearing and scaling in addition to the rotation and translation. This is usually a good choice of transform for initialization of non-rigid transforms like the B-Spline transform. 
- **Non-rigid registration** methods are capable of aligning images where correspondence cannot be achieved without localized deformations and can therefore better accomodate anatomical, physiological and pathological variability between patients.
- **Groupwise registration** methods try to mitigate uncertainties associated with any one image by simultaneously registering all images in a population.
- **Point-based registration** allows us to help the registration via pre-defined sets of corresponding points. 

In [121]:
# Let's save a quick image to make sure we know what plane we are in.
# Default is the Saggittal Plane when we call ReadImage() on our .mhd files
temp_img = sitk.ReadImage(TOY_LUNA_PATH + mhd_one)
sitk.WriteImage(temp_img[300], IMG_SAVE+'result5.tif')

In [154]:
# lets try registering two scans after normalizing each scan.

def normalize_img(img):
    """
    Sets the MHD image to be approximately 1.0 mm voxel size
    https://itk.org/ITKExamples/src/Filtering/ImageGrid/ResampleAnImage/Documentation.html
    """
    # Number of pixels you want for x,y,z dimensions
    new_x_size = int(img.GetSpacing()[0]*img.GetWidth())
    new_y_size = int(img.GetSpacing()[1]*img.GetHeight())
    new_z_size = int(img.GetSpacing()[2]*img.GetDepth())
    new_size = [new_x_size, new_y_size, new_z_size]
    new_spacing = [1,1,1]  # New spacing to be 1.0 x 1.0 x 1.0 mm voxel size

    interpolator_type = sitk.sitkBSpline #sitkLinear using BSpline over Linear
    return sitk.Resample(img, np.array(new_size, dtype='uint32').tolist(),
                sitk.Transform(),
                interpolator_type,
                img.GetOrigin(),
                new_spacing,
                img.GetDirection(),
                0.0,
                img.GetPixelIDValue())


# Load the CT scan (3D .mhd file)
# Numpy is z,y,x and SimpleITK is x,y,z -- (note the ordering of dimesions)
itk_img_one = sitk.ReadImage(TOY_LUNA_PATH + mhd_one)
itk_img_two = sitk.ReadImage(TOY_LUNA_PATH + mhd_two)

# Normalize the image spacing so that a voxel is 1x1x1 mm in dimension
itk_img_one = normalize_img(itk_img_one)
itk_img_two = normalize_img(itk_img_two)

# # Register both CT scans post normalization()
resultImage = sitk.Elastix(itk_img_one,itk_img_two,"translation") #simple translation register

sitk.WriteImage(resultImage[150], IMG_SAVE+'result7.tif')

In [155]:
# Try writing each image taking a different slice
sitk.WriteImage(resultImage[150], IMG_SAVE+'result8_baseline.tif')
sitk.WriteImage(resultImage[150,:,:], IMG_SAVE+'result8a.tif')
sitk.WriteImage(resultImage[:,150,:], IMG_SAVE+'result8b.tif')
sitk.WriteImage(resultImage[:,:,150], IMG_SAVE+'result8c.tif')

In [153]:
# Write baseline images corresponding to img8 pre register
sitk.WriteImage(itk_img_one[150], IMG_SAVE+'result9_baseline.tif')
sitk.WriteImage(itk_img_one[150,:,:], IMG_SAVE+'result9a.tif')
sitk.WriteImage(itk_img_one[:,150,:], IMG_SAVE+'result9b.tif')
sitk.WriteImage(itk_img_one[:,:,150], IMG_SAVE+'result9c.tif')

In [171]:
# Let's examine what setting the fixed and moving image looks like

fixedImage = sitk.ReadImage(TOY_LUNA_PATH + mhd_two)
movingImage = sitk.ReadImage(TOY_LUNA_PATH + mhd_one)
parameterMap = sitk.GetDefaultParameterMap('translation')

elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(fixedImage)
elastixImageFilter.SetMovingImage(movingImage)
elastixImageFilter.SetParameterMap(parameterMap)
elastixImageFilter.Execute()

# resultImage = elastixImageFilter.GetResultImage()
# transformParameterMap = elastixImageFilter.GetTransformParameterMap()
# sitk.WriteImage(resultImage[300], IMG_SAVE+'result2.tif')

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x184784f6f0> >

In [174]:
def myshow3d(img, xslices=[], yslices=[], zslices=[], title=None, margin=0.05, dpi=80):
    size = img.GetSize()
    img_xslices = [img[s,:,:] for s in xslices]
    img_yslices = [img[:,s,:] for s in yslices]
    img_zslices = [img[:,:,s] for s in zslices]
    
    maxlen = max(len(img_xslices), len(img_yslices), len(img_zslices))
    
        
    img_null = sitk.Image([0,0], img.GetPixelIDValue(), img.GetNumberOfComponentsPerPixel())
    
    img_slices = []
    d = 0
    
    if len(img_xslices):
        img_slices += img_xslices + [img_null]*(maxlen-len(img_xslices))
        d += 1
        
    if len(img_yslices):
        img_slices += img_yslices + [img_null]*(maxlen-len(img_yslices))
        d += 1
     
    if len(img_zslices):
        img_slices += img_zslices + [img_null]*(maxlen-len(img_zslices))
        d +=1
    
    if maxlen != 0:
        if img.GetNumberOfComponentsPerPixel() == 1:
            img = sitk.Tile(img_slices, [maxlen,d])
            return img
        #TODO check in code to get Tile Filter working with VectorImages
        else:
            img_comps = []
            for i in range(0,img.GetNumberOfComponentsPerPixel()):
                img_slices_c = [sitk.VectorIndexSelectionCast(s, i) for s in img_slices]
                img_comps.append(sitk.Tile(img_slices_c, [maxlen,d]))
            img = sitk.Compose(img_comps)
            return img
            
    
#     myshow(img, title, margin, dpi)

In [175]:
size = resultImage.GetSize()
myshow3d(resultImage,yslices=range(50,size[1]-50,30), zslices=range(50,size[2]-50,20), dpi=30)


<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x184697b5d0> >

In [158]:
resultImage.GetSize()

(275, 275, 341)

In [166]:
def myshow(img, title=None, margin=0.05, dpi=80 ):
    nda = sitk.GetArrayFromImage(img)
    spacing = img.GetSpacing()
    
    
    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]
        
        # the the number of components is 3 or 4 consider it an RGB image
        if not c in (3,4):
            nda = nda[nda.shape[0]//2,:,:]
    
    elif nda.ndim == 4:
        c = nda.shape[-1]
        
        if not c in (3,4):
            raise Runtime("Unable to show 3D-vector Image")
            
        # take a z-slice
        nda = nda[nda.shape[0]//2,:,:,:]
            
    ysize = nda.shape[0]
    xsize = nda.shape[1]
   
    
    # Make a figure big enough to accomodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi

    fig = figure(figsize=figsize, dpi=dpi)
    # Make the axis the right size...
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    
    extent = (0, xsize*spacing[1], ysize*spacing[0], 0)
    
    t = ax.imshow(nda,extent=extent,interpolation=None)
    
    if nda.ndim == 2:
        t.set_cmap("gray")
    
    if(title):
        plt.title(title)
    
slices =[resultImage[size[0]//2,:,:], resultImage[:,size[1]//2,:], resultImage[:,:,size[2]//2]]
myshow(sitk.Tile(slices, [3,1]), dpi=20)

NameError: name 'figure' is not defined

In [146]:
print(len(resultImage))


25788125


In [139]:
print(itk_img)

Image (0x7feabdf05a20)
  RTTI typeinfo:   itk::Image<short, 3u>
  Reference Count: 1
  Modified Time: 440089
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 440063
  UpdateMTime: 440087
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [275, 275, 341]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [275, 275, 341]
  RequestedRegion: 
    Dimension: 3
    Index: [10, 0, 0]
    Size: [1, 275, 341]
  Spacing: [1, 1, 1]
  Origin: [-130.231, -268.231, -405]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
1 0 0
0 1 0
0 0 1

  PointToIndexMatrix: 
1 0 0
0 1 0
0 0 1

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x7feabdf06be0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, short>
      Reference Count: 1
      Modified Time:

In [106]:
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(sitk.ReadImage(TOY_LUNA_PATH + mhd_one)[1])
elastixImageFilter.SetMovingImage(sitk.ReadImage(TOY_LUNA_PATH + mhd_two)[1])

parameterMapVector = sitk.VectorOfParameterMap()
parameterMapVector.append(sitk.GetDefaultParameterMap("affine"))
parameterMapVector.append(sitk.GetDefaultParameterMap("bspline"))
elastixImageFilter.SetParameterMap(parameterMapVector)

elastixImageFilter.Execute() #execute register procedure



<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x182541a5a0> >

In [107]:
result = elastixImageFilter.GetResultImage()
print(result)

Image (0x7feabdf0cbe0)
  RTTI typeinfo:   itk::Image<float, 2u>
  Reference Count: 3
  Modified Time: 171175
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 0
  UpdateMTime: 0
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [512, 682]
  BufferedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [512, 682]
  RequestedRegion: 
    Dimension: 2
    Index: [0, 0]
    Size: [512, 682]
  Spacing: [0.537109, 0.5]
  Origin: [-268.231, -405]
  Direction: 
1 0
0 1

  IndexToPointMatrix: 
0.537109 0
0 0.5

  PointToIndexMatrix: 
1.86182 0
0 2

  Inverse Direction: 
1 0
0 1

  PixelContainer: 
    ImportImageContainer (0x7feabdcabb40)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, float>
      Reference Count: 1
      Modified Time: 171176
      Debug: Off
      Object Name: 
      Observers: 

In [108]:
print(len(result))

349184


In [110]:
#save only 2d slice.
sitk.WriteImage(result, IMG_SAVE+'result1.tif')

In [ ]:
# Concatenate the ND images into one (N+1)D image
population = ['image1.hdr', ..., 'imageN.hdr']
vectorOfImages = sitk.VectorOfImage()

for filename in population
  vectorOfImages.push_back(sitk.ReadImage(filename))

image = sitk.JoinSeries(vectorOfImages)

# Register
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(image)
elastixImageFilter.SetMovingImage(image)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap('groupwise'))
elastixImageFilter.Execute()

In [70]:
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return ct_scan, origin, spacing
t = load_itk(TOY_LUNA_PATH + mhd)
t

(array([[[-1002,  -982,  -968, ...,  -972,  -985,  -976],
         [-1000, -1008,  -996, ..., -1001,  -986,  -980],
         [ -975, -1017, -1024, ...,  -991,  -995, -1011],
         ...,
         [ -921,  -925,  -918, ..., -1016, -1008,  -998],
         [ -951,  -916,  -898, ..., -1024, -1011, -1015],
         [ -938,  -893,  -876, ...,  -995,  -996, -1019]],
 
        [[-1021, -1012, -1007, ...,  -995, -1015,  -988],
         [-1022, -1003,  -996, ...,  -994, -1001,  -987],
         [-1019, -1003, -1000, ..., -1009,  -999,  -995],
         ...,
         [ -910,  -922,  -924, ...,  -999, -1023, -1021],
         [ -934,  -930,  -922, ..., -1009, -1023, -1023],
         [ -932,  -927,  -910, ..., -1010,  -993,  -994]],
 
        [[-1009, -1007,  -997, ...,  -975, -1000, -1020],
         [ -988,  -988,  -993, ..., -1006, -1014, -1018],
         [ -992,  -974,  -989, ..., -1024, -1021, -1017],
         ...,
         [ -907,  -882,  -876, ...,  -967,  -980,  -987],
         [ -903,  -890, 